In [32]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Projects/mwp_enhanced/modeling2
# Install and Load Libraries

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Projects/mwp_enhanced/modeling2


# Install and Load Libraries




In [33]:

!pip install yahoo-fin
!pip install hvplot
!pip install pandas-ta
# url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
# !curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
# url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py39hd257fcd_4.tar.bz2'
# !curl -L $url | tar xj -C /usr/local/lib/python3.9/dist-packages/ lib/python3.9/site-packages/talib --strip-components=3
!pip install pycaret
!pip install pycaret[tuners]
!pip install "schemdraw<0.16"
!pip install mlflow --quiet
!pip install mplfinance
!pip install python-dotenv
# import modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import panel as pn
pn.extension('tabulator')
import pandas as pd

from panel.template import FastListTemplate
from pathlib import Path
from yahoo_fin.stock_info import get_data
import datetime
from matplotlib.figure import Figure
from matplotlib import cm
%matplotlib inline

# import modules that help build tabs
import modules.helpers as helpers
import modules.HistoricalData as hst
import modules.MCTab as MCTab
import modules.intro as intro
import modules.profile as prf
import modules.AlgoTab as at

import pandas_ta as ta
# import talib
# from yahoo_finance_api2 import share
import numpy as np
# import mplfinance as mpf

from joblib import dump, load
from pycaret.classification import *

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import mlflow
from getpass import getpass
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

from sklearn.metrics import brier_score_loss
from sklearn.ensemble import HistGradientBoostingClassifier
!pip install scikit-optimize
from skopt import BayesSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

load_dotenv()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


True

# Load datasets

In [34]:
X_train = pd.read_csv(Path("./data/prepared/Xtrain.csv"))
X_test = pd.read_csv(Path("./data/prepared/Xtest.csv"))
y_train = pd.read_csv(Path("./data/prepared/ytrain.csv"))
y_test = pd.read_csv(Path("./data/prepared/ytest.csv"))
X_validation = pd.read_csv(Path("./data/prepared/Xvalidation.csv"))
y_validation = pd.read_csv(Path("./data/prepared/yvalidation.csv"))

In [35]:
mlflow.set_tracking_uri(f"https://dagshub.com/tlchampion/map_trials.mlflow")
# mlflow.set_experiement("HistGradBoost")
# mlflow.set_tag("optimization")


# Optimize Gradient Boosted Classifier

In [ ]:
gbc = ClassificationExperiment()
gbc.setup(X_train ,
             target = y_train.squeeze(),
             session_id = 123, 
             fix_imbalance=False, 
             normalize=True, 
             normalize_method='minmax',
             use_gpu = False,
             log_experiment = True,
             experiment_name = "gbc_tuning",
             log_plots = True)
gbc.add_metric("brier", "Brier Score", brier_score_loss, greater_is_better = False, target="pred_proba")

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(64124, 33)"
4,Transformed data shape,"(64124, 33)"
5,Transformed train set shape,"(44886, 33)"
6,Transformed test set shape,"(19238, 33)"
7,Numeric features,32
8,Preprocess,True
9,Imputation type,simple


Name                                                       Brier Score
Display Name                                               Brier Score
Score Function           <function brier_score_loss at 0x7f55497d2200>
Scorer               make_scorer(brier_score_loss, greater_is_bette...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: brier, dtype: object

In [ ]:
gbc_model = gbc.create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Brier Score
Fold,,,,,,,,
0,0.5415,0.5468,0.8732,0.5442,0.6705,0.0360,0.0490,0.2460
1,0.5420,0.5334,0.8816,0.5440,0.6728,0.0358,0.0499,0.2463
2,0.5266,0.5262,0.8319,0.5367,0.6525,0.0088,0.0111,0.2475
3,0.5429,0.5522,0.8749,0.5449,0.6716,0.0387,0.0529,0.2450
4,0.5375,0.5434,0.8657,0.5420,0.6667,0.0281,0.0379,0.2458
5,0.5382,0.5479,0.8707,0.5422,0.6683,0.0289,0.0394,0.2453
6,0.5385,0.5529,0.8715,0.5423,0.6686,0.0297,0.0406,0.2457
7,0.5363,0.5416,0.8624,0.5415,0.6653,0.0257,0.0344,0.2466
8,0.5332,0.5360,0.8474,0.5403,0.6598,0.0210,0.0272,0.2464


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
gbc_tuned, gbc_tuner  = gbc.tune_model(gbc_model, return_tuner=True, optimize='brier', search_library="tune-sklearn", search_algorithm="hyperopt", n_iter=25)

2023-05-08 16:44:26,590	WARNING util.py:244 -- The `reset` operation took 0.616 s, which may be a performance bottleneck.
2023-05-08 16:44:26,606	WARNING util.py:244 -- The `start_trial` operation took 0.634 s, which may be a performance bottleneck.
2023-05-08 17:17:20,972	WARNING util.py:244 -- The `reset` operation took 0.684 s, which may be a performance bottleneck.
2023-05-08 17:17:20,985	WARNING util.py:244 -- The `start_trial` operation took 0.699 s, which may be a performance bottleneck.
2023-05-08 19:13:43,973	WARNING util.py:244 -- The `reset` operation took 0.663 s, which may be a performance bottleneck.
2023-05-08 19:13:43,984	WARNING util.py:244 -- The `start_trial` operation took 0.676 s, which may be a performance bottleneck.
2023-05-08 19:27:12,993	INFO tune.py:945 -- Total run time: 20273.36 seconds (20266.75 seconds for the tuning loop).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Brier Score
Fold,,,,,,,,
0,0.5313,0.5282,0.9012,0.5365,0.6726,0.0087,0.0136,0.2463
1,0.5404,0.5309,0.9133,0.5414,0.6798,0.0275,0.0437,0.2464
2,0.5353,0.5312,0.8924,0.5393,0.6723,0.0191,0.0283,0.2465
3,0.5436,0.5463,0.9112,0.5434,0.6808,0.0349,0.0541,0.2454
4,0.5349,0.5382,0.8974,0.5388,0.6733,0.0173,0.0263,0.2456
5,0.5400,0.5417,0.9003,0.5418,0.6765,0.0284,0.0428,0.2458
6,0.5345,0.5414,0.8957,0.5386,0.6727,0.0170,0.0257,0.2455
7,0.5374,0.5480,0.8978,0.5404,0.6747,0.0229,0.0344,0.2457
8,0.5385,0.5358,0.8945,0.5413,0.6744,0.0259,0.0382,0.2461


In [ ]:
print(gbc_tuner)

TuneSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             early_stopping=False, error_score=nan,
             estimator=Pipeline(memory=FastMemory(location=/tmp/joblib),
                                steps=[('numerical_imputer',
                                        TransformerWrapper(exclude=None,
                                                           include=['RSI_5',
                                                                    'RSI_10',
                                                                    'RSI_30',
                                                                    'RSI_60',
                                                                    '5day_avg_ret',
                                                                    '10day_avg_ret',
                                                                    '30day_avg_ret',
                                                                    '60day_avg_ret',
           

In [ ]:
print(gbc_tuned)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.003944321219069253, loss='log_loss',
                           max_depth=10, max_features=0.886727525845163,
                           max_leaf_nodes=None,
                           min_impurity_decrease=0.12030146051293354,
                           min_samples_leaf=1, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=287,
                           n_iter_no_change=None, random_state=123,
                           subsample=0.9265771876529256, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [ ]:
save_model(gbc_tuned,Path("./pycaret_models/gbc_tuned"))

ValueError: ignored

In [ ]:
plot_model(gbc_tuned, plot='auc')

ValueError: ignored

In [ ]:
plot_model(gbc_tuned, plot='confusion_matrix')

# LightGBM

In [ ]:
lgbm = ClassificationExperiment()
lgbm.setup(X_train ,
             target = y_train.squeeze(),
             session_id = 123, 
             fix_imbalance=False, 
             normalize=True, 
             normalize_method='minmax',
             use_gpu = False,
             log_experiment = True,
             experiment_name = "lgbm_tuning",
             log_plots = True)
lgbm.add_metric("brier", "Brier Score", brier_score_loss, greater_is_better = False, target="pred_proba")

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(64124, 33)"
4,Transformed data shape,"(64124, 33)"
5,Transformed train set shape,"(44886, 33)"
6,Transformed test set shape,"(19238, 33)"
7,Numeric features,32
8,Preprocess,True
9,Imputation type,simple


Name                                                       Brier Score
Display Name                                               Brier Score
Score Function           <function brier_score_loss at 0x7f55497d2200>
Scorer               make_scorer(brier_score_loss, greater_is_bette...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: brier, dtype: object

In [ ]:
lgbm_model = lgbm.create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Brier Score
Fold,,,,,,,,
0,0.5369,0.5436,0.7590,0.5480,0.6365,0.0423,0.0468,0.2460
1,0.5353,0.5323,0.7560,0.5471,0.6348,0.0393,0.0434,0.2475
2,0.5222,0.5251,0.7227,0.5394,0.6177,0.0153,0.0165,0.2482
3,0.5496,0.5540,0.7752,0.5563,0.6477,0.0680,0.0755,0.2451
4,0.5329,0.5444,0.7515,0.5456,0.6322,0.0346,0.0380,0.2460
5,0.5420,0.5535,0.7590,0.5518,0.6390,0.0536,0.0589,0.2455
6,0.5377,0.5461,0.7580,0.5486,0.6365,0.0443,0.0488,0.2464
7,0.5343,0.5431,0.7452,0.5472,0.6310,0.0386,0.0421,0.2464
8,0.5432,0.5443,0.7565,0.5530,0.6390,0.0566,0.0619,0.2466


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
lgbm_tuned, lgbm_tuner  = lgbm.tune_model(lgbm_model, return_tuner=True, optimize='brier', search_library="tune-sklearn", search_algorithm="hyperopt", n_iter=25)

(_Trainable pid=108288) [LightGBM] [Warning] bagging_fraction is set=0.8175075296235957, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8175075296235957
(_Trainable pid=108288) [LightGBM] [Warning] feature_fraction is set=0.9681460404949997, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9681460404949997
(_Trainable pid=108288) [LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
(_Trainable pid=108288) [LightGBM] [Warning] bagging_fraction is set=0.8175075296235957, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8175075296235957
(_Trainable pid=108288) [LightGBM] [Warning] feature_fraction is set=0.9681460404949997, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9681460404949997
(_Trainable pid=108288) [LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
(_Trainable pid=108353) [LightGBM] [Wa

2023-05-08 20:24:47,716	WARNING util.py:244 -- The `reset` operation took 0.924 s, which may be a performance bottleneck.
2023-05-08 20:24:47,728	WARNING util.py:244 -- The `start_trial` operation took 0.937 s, which may be a performance bottleneck.


(_Trainable pid=108288) [LightGBM] [Warning] bagging_fraction is set=0.5318364984714008, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5318364984714008 [repeated 2x across cluster]
(_Trainable pid=108288) [LightGBM] [Warning] feature_fraction is set=0.7035424522979642, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7035424522979642 [repeated 3x across cluster]
(_Trainable pid=108288) [LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6 [repeated 3x across cluster]
(_Trainable pid=108353) [LightGBM] [Warning] bagging_fraction is set=0.7078750819469858, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7078750819469858 [repeated 2x across cluster]
(_Trainable pid=108353) [LightGBM] [Warning] feature_fraction is set=0.6682470001755085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6682470001755085 [repeated 2x across cluster]
(_Trainable pid=108353) [

2023-05-08 20:27:08,870	WARNING util.py:244 -- The `reset` operation took 0.544 s, which may be a performance bottleneck.
2023-05-08 20:27:08,883	WARNING util.py:244 -- The `start_trial` operation took 0.558 s, which may be a performance bottleneck.


(_Trainable pid=108353) [LightGBM] [Warning] bagging_fraction is set=0.5537832335991402, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5537832335991402
(_Trainable pid=108353) [LightGBM] [Warning] feature_fraction is set=0.9378054320101126, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9378054320101126
(_Trainable pid=108353) [LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
(_Trainable pid=108288) [LightGBM] [Warning] bagging_fraction is set=0.8915083515846672, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8915083515846672
(_Trainable pid=108288) [LightGBM] [Warning] feature_fraction is set=0.9513082229031942, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9513082229031942
(_Trainable pid=108288) [LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
(_Trainable pid=108288) [LightGBM] [Wa

...

2023-05-08 20:37:44,618	WARNING util.py:244 -- The `reset` operation took 1.677 s, which may be a performance bottleneck.
2023-05-08 20:37:44,628	WARNING util.py:244 -- The `start_trial` operation took 1.688 s, which may be a performance bottleneck.


(_Trainable pid=108288) [LightGBM] [Warning] bagging_fraction is set=0.7336935972650009, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7336935972650009 [repeated 3x across cluster]
(_Trainable pid=108288) [LightGBM] [Warning] feature_fraction is set=0.6386161241316964, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6386161241316964 [repeated 3x across cluster]
(_Trainable pid=108288) [LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5 [repeated 3x across cluster]
(_Trainable pid=108288) [LightGBM] [Warning] bagging_fraction is set=0.7336935972650009, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7336935972650009 [repeated 2x across cluster]
(_Trainable pid=108288) [LightGBM] [Warning] feature_fraction is set=0.6386161241316964, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6386161241316964 [repeated 2x across cluster]
(_Trainable pid=108288) [

2023-05-08 20:39:02,541	INFO tune.py:945 -- Total run time: 922.82 seconds (922.75 seconds for the tuning loop).


[LightGBM] [Warning] bagging_fraction is set=0.7078750819469858, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7078750819469858
[LightGBM] [Warning] feature_fraction is set=0.6682470001755085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6682470001755085
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Brier Score
Fold,,,,,,,,
0,0.5369,0.5450,0.8224,0.5440,0.6548,0.0331,0.0403,0.2453
1,0.5438,0.5429,0.8369,0.5478,0.6622,0.0463,0.0573,0.2456
2,0.5309,0.5331,0.8128,0.5405,0.6492,0.0211,0.0255,0.2465
3,0.5373,0.5510,0.8198,0.5444,0.6544,0.0344,0.0417,0.2448
4,0.5366,0.5466,0.8086,0.5447,0.6509,0.0346,0.0411,0.2447
5,0.5411,0.5463,0.8219,0.5469,0.6568,0.0425,0.0514,0.2454
6,0.5434,0.5555,0.8290,0.5480,0.6598,0.0469,0.0571,0.2446
7,0.5357,0.5518,0.8090,0.5440,0.6506,0.0322,0.0383,0.2451
8,0.5343,0.5382,0.8115,0.5430,0.6506,0.0288,0.0345,0.2460


In [ ]:
print(lgbm_tuner)

TuneSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             early_stopping=False, error_score=nan,
             estimator=Pipeline(memory=FastMemory(location=/tmp/joblib),
                                steps=[('numerical_imputer',
                                        TransformerWrapper(exclude=None,
                                                           include=['RSI_5',
                                                                    'RSI_10',
                                                                    'RSI_30',
                                                                    'RSI_60',
                                                                    '5day_avg_ret',
                                                                    '10day_avg_ret',
                                                                    '30day_avg_ret',
                                                                    '60day_avg_ret',
           

In [ ]:
print(lgbm_tuned)

LGBMClassifier(bagging_fraction=0.7078750819469858, bagging_freq=2,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.6682470001755085, importance_type='split',
               learning_rate=0.011908256456919489, max_depth=-1,
               min_child_samples=62, min_child_weight=0.001,
               min_split_gain=0.6870042099013675, n_estimators=209, n_jobs=-1,
               num_leaves=62, objective=None, random_state=123,
               reg_alpha=1.5754585952727133e-07,
               reg_lambda=8.304171871667417e-08, silent='warn', subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
